# Partial Equivalence Checking of Quantum Circuits

## Partial Equivalence vs. Total Equivalence

Different definitions of quantum circuit equivalence have been proposed, with the most commonly used being total equivalence. Two circuits are considered totally equivalent when their matrix representations are exactly the same. However, it is often sufficient to consider observational equivalence, because the only way to extract information from a quantum circuit is through measurement. Therefore, partial equivalence has been defined, which is a weaker equality than total equivalence. Two circuits are considered partially equivalent if, for each possible input state, they have the same probabilities for each measurement outcome. 

This definition is especially useful for circuits that do not measure all of their qubits at the end, or where some qubits are initialized to a certain value at the beginning. We refer to the qubits that are initialized to a certain value at the beginning of the computation *ancillary* qubits. Without loss of generality, we assume that these qubits are initially set to $|0\rangle$. Any other initial state can be reached from the $|0\rangle$ state by applying the appropriate gates. The remaining qubits are the qubits that hold the input state, and they are referred to as *data* qubits.

Similarly, we differentiate between *measured* and *garbage* qubits. In order to read the output at the end of a circuit, a measurement is performed on some qubits, but not necessarily all of them. These qubits are the *measured* qubits. All qubits that are not measured are called the *garbage* qubits. Their final state doesn't influence the output we obtain from the computation. 

For a circuit $C$ we denote $P(t | \psi, C)$ as the probability that the quantum circuit $C$ collapses to state $|t\rangle$ upon a measurement on the measured qubits, given that the data qubits have the initial state $|\psi\rangle$.

With these concepts we can provide an exact definition of partial equivalence. Two circuits $C_1$ and $C_2$ are considered partially equivalent if, for each initial state $|\psi\rangle$ of the data qubits and each final state $|t\rangle$ of the measured qubits, it holds that $P(t|\psi, C_1) = P(t|\psi, C_2)$.



## Equivalence Checking of Partially Equivalent Quantum Circuits

Partial equivalence checking is a bit more costly than regular equivalence checking. Therefore, it is not enabled by default in QCEC. It can be activated using an option in the configuration parameters. If the option `check_partial_equivalence` is set to `True`, the equivalence checker will return `equivalent` not only for totally equivalent circuits, but also for partially equivalent circuits. The result will be `not_equivalent` if and only if the circuits are not partially equivalent.

The following is a summary of the behaviour of each type of equivalence checker when the `check_partial_equivalence` option is set to `True`.

1. **Construction Equivalence Checker:** The construction checker supports partial equivalence checking by using decision diagrams to calculate the measurement probabilities of the possible ouputs and it compares the results for the two circuits.

1. **Alternating Equivalence Checker:** The alternating checker can only be used for circuits where at least one of the two does not contain ancillary qubits. It normally computes the composition of one circuit with the inverse of the other and verifies that the result is the identity matrix. When checking for partial equivalence, it suffices to verify that the result is a diagonal matrix (modulo garbage qubits). This check doesn't have a signifi

1. **Simulation Equivalence Checker:** The simulation checker ...

1. **ZX-Calculus Equivalence Checker:** The ZX-calculus checker doesn't directly support partial equivalence, which is not a problem for the equivalence checking workflow, given that the ZX-calculus checker cannot demonstrate non-equivalence of circuits. Therefore it will simply output 'No Information' for circuits that are partially but not totally equivalent.

## Using QCEC to Verify Partial Equivalent Circuits

Consider the following quantum circuit with three qubits, which are all data qubits, but only one of them is measured.

In [ ]:
from qiskit import QuantumCircuit

qc_lhs = QuantumCircuit(3, 1)
qc_lhs.cswap(1, 0, 2)
qc_lhs.h(0)
qc_lhs.z(2)
qc_lhs.cswap(1, 0, 2)
qc_lhs.measure(0, 0)

qc_lhs.draw(output="mpl", style="iqp")

The following circuit is not totally equivalent to the previous one. The two circuits are however partially equivalent, because they have the same probability distribution of measured values for any given initial input state.

In [ ]:
qc_rhs = QuantumCircuit(3, 1)
qc_rhs.x(1)
qc_rhs.ch(1, 0)
qc_rhs.measure(0, 0)
qc_rhs.draw(output="mpl", style="iqp")

This equality can be proved with QCEC by using the `verify` method and setting the configuration `check_partial_equivalence` to `True`:

In [ ]:
from mqt import qcec

qcec.verify(qc_lhs, qc_rhs, check_partial_equivalence=True)

Source: The definitions and algorithm for partial equivalence are described in the paper [Partial Equivalence Checking of Quantum Circuits](https://arxiv.org/abs/2208.07564) by Tian-Fu Chen, Jie-Hong R. Jiang and Min-Hsiu Hsieh